In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# --- Config Selenium ---
driver = webdriver.Chrome()  # ou webdriver.Firefox()
driver.get("https://www.ecologie.gouv.fr/politiques-publiques/prix-produits-petroliers")

# --- Attendre que le tableau soit chargé ---
wait = WebDriverWait(driver, 10)

# Repérer le h3 correspondant au tableau voulu
h3_element = wait.until(EC.presence_of_element_located(
    (By.XPATH, '//h3[contains(text(), "Évolution des coûts moyens de transport-distribution en France en c€/l depuis 2015")]')
))

# Le tableau est juste après le h3
table_element = h3_element.find_element(By.XPATH, 'following-sibling::div[@class="fr-table fr-table--blue-ecume"]//table')

# Lire les lignes du tableau
rows = table_element.find_elements(By.TAG_NAME, "tr")

data = []
for row in rows[1:]:  # skip header
    cols = row.find_elements(By.TAG_NAME, "td")
    year = int(cols[0].text.strip())
    gazole = float(cols[1].text.strip().replace(",", "."))
    data.append([year, gazole])

# Créer DataFrame
df_costs = pd.DataFrame(data, columns=["Year", "Gazole_transportation_fees_eur_liter"])

# Fermer le driver
driver.quit()

# Créer un dataframe de YYYY-MM
df_monthly = pd.DataFrame({"Date_monthly": pd.date_range(start=f"{df_costs['Year'].min()}-01-01",
                                                         end=f"{df_costs['Year'].max()}-12-01",
                                                         freq='MS')})
df_monthly['Year'] = df_monthly['Date_monthly'].dt.year

# Merger avec le df des frais de transport
df_monthly_costs = df_monthly.merge(df_costs, on='Year', how='left')
df_monthly_costs = df_monthly_costs.drop(columns=['Year'])

# Export CSV
os.makedirs("csv", exist_ok=True)
df_monthly_costs.to_csv("csv/transport_fees_eur_liter.csv", index=False)

print(df_monthly_costs)

NameError: name 'os' is not defined